In [1]:
%load_ext autoreload
%autoreload 1
%aimport ImageClassifier
%aimport model_runner
%aimport ConvNet
%aimport Timer
%matplotlib inline

ImageClassifier Loaded!!!


In [2]:
from ImageClassifier import ImageClassifier as IC
from ImageClassifier import ModelTrainer as MT
from Timer import Timer
from model_runner import *
import ConvNet as CNN
from matplotlib import pyplot

def basicNorm(img):
    shape = img.shape
    dst = np.float32(img)
    if (len(shape) == 2):
            p  = dst
            p -= p.min()
            p /= p.max()        
    else:
        for i in range(shape[2]): 
            p  = dst[:, :, i]
            p -= p.min()
            p /= p.max()

    return dst
            
def visualize(data, setname='train'):
    
    dataset = data.dmap[setname]
    signnames = data.signs
    col_width = max(len(name) for name in signnames)
    class_counts = data.get_data_dist(setname)
    
    for c in range(len(signnames)):
        X_train = dataset[c]
        c_count = len(X_train)
        c_index = 0
        
        print("Class %i: %-*s  %s samples" % (c, col_width, signnames[c], str(c_count)))
        fig = pyplot.figure(figsize = (6, 1))
        fig.subplots_adjust(left = 0, right = 1, bottom = 0, top = 1, hspace = 0.05, wspace = 0.05)
        random_indices = random.sample(range(c_index, c_index + c_count), 10)
        for i in range(10):
            axis = fig.add_subplot(1, 10, i + 1, xticks=[], yticks=[])
            axis.imshow(basicNorm(X_train[random_indices[i]]))
                
        pyplot.show()
        print("--------------------------------------------------------------------------------------\n")

    pyplot.bar( np.arange( 43 ), class_counts, align='center' )
    pyplot.xlabel('Class')
    pyplot.ylabel('Number of training examples')
    pyplot.xlim([-1, 43])
    pyplot.show()

## Data Load + Augmentation

In [3]:
import os
import pickle

def load(dirpath='traffic-signs-data'):
    org  = IC(data_dir=dirpath)
    data = DataModifier(org)
    return org, data

def saveData(org, dirpath):
    dirpath = Utils.find_an_empty_dir(dirpath)
    print("Saving data to: " + dirpath)
    if (os.path.exists(dirpath) == False): os.makedirs(dirpath)
    with open(os.path.join(dirpath, 'train.p'), "wb") as fp:
        pickle.dump(org.train, fp)
    with open(os.path.join(dirpath, 'test.p'), "wb" ) as fp:
        pickle.dump(org.test, fp)
    with open(os.path.join(dirpath, 'valid.p'), "wb") as fp:
        pickle.dump(org.valid, fp)

def augmentData(org=None, data=None, save_dir=None, num_total=1000):
#     augmentDataSet(data.train)
#     augmentDataSet(data.test)
    n_train,n_test,n_valid = num_total, np.int(num_total*0.3), np.int(num_total*0.1)
    augmentDatasetPerspective(data.train, num_total=n_train, sz=26, delta=3, t_rng=range(-3, 3), perspTrans=True)
    augmentDatasetPerspective(data.test,  num_total=n_test , sz=26, delta=3, t_rng=range(-3, 3), perspTrans=True)
    augmentDatasetPerspective(data.valid, num_total=n_valid, sz=26, delta=3, t_rng=range(-3, 3), perspTrans=True)
    DataModifier.updateDataSet(org, data.train, org.train)
    DataModifier.updateDataSet(org, data.test,  org.test)
    DataModifier.updateDataSet(org, data.valid, org.valid)    
    if (save_dir is not None): saveData(org, save_dir)
    return org, data

# for n in range(3):
#     for i in [1, 2, 3]:
#         org, data = Timer.run(load)
#         org, data = Timer.run(augmentData, args={'org': org, 'data': data, 'num_total': i*1000, 'save_dir':'aug{:1d}K_sz26_d3_rng3_persp'.format(i)})





## Load Augmented Data

In [39]:
data_path = 'data/aug3K_sz26_d3_rng3_persp/000'
org, data = Timer.run(load, args={'dirpath': data_path})
# org, data = Timer.run(load)


---------------------------------
      | Train  | Test   | Valid  |
---------------------------------
 (#)  | 129000 |  38700 |  12900 |
 (%)  |   1.00 |   0.30 |   0.10 |
---------------------------------
Number Classes  : 43
Image Dimensions: (32, 32, 3)

---------------------------------------------------------------
elapsed time: 2.668 secs
---------------------------------------------------------------


## Reset / Visualize

In [40]:
# org.reset_data()
# visualize(data)


## Preprocess

---------------------------------------------------------------
elapsed time: 7.170 secs
---------------------------------------------------------------


## Train

In [53]:
def preprocess(org=None, data=None, pre_ops=[]):
    if (len(pre_ops) > 0): org.preprocess_all(pre_ops)
    return org, data


def Center_1to1(src, i, dst):
    dst[i] = (src[i] - 128.0)/128.0

kEPOCHS, kBATCH_SIZE, kRATE, kNETWORK, kNETWORK_ARGS = 'EPOCHS', 'BATCH_SIZE', 'RATE', 'NETWORK', 'NETWORK_ARGS'

def train(D=None, EPOCHS=10, BATCH_SIZE=128, RATE=0.001, NETWORK=MT.LeNet, NETWORK_ARGS={}):
    if D is None: raise Exception('D in None')
    T = MT(D, EPOCHS=EPOCHS, BATCH_SIZE=BATCH_SIZE, rate=RATE, network=NETWORK, network_args=NETWORK_ARGS)
    T.train()
    return D, T

# pre_ops = [Center_1to1]
pre_ops = [IC.Norm, IC.ZMean]
org, data = Timer.run(preprocess,  args={'org': org, 'data': data, 'pre_ops':pre_ops})

# , 'dropouts':{0: 0.9, 1:0.6}
args = {'D': org, 'EPOCHS': 25, 'BATCH_SIZE': 64, 'RATE': 0.0001, 'NETWORK': MT.LeNetWithDropOut, 'NETWORK_ARGS': {'sigma':0.001}} 

org, modl = Timer.run(train, args=args)


ImageClassifier Loaded!!!
---------------------------------------------------------------
elapsed time: 7.205 secs
---------------------------------------------------------------
Training...

EPOCH  1 ...
Validation Accuracy = 0.673

EPOCH  2 ...
Validation Accuracy = 0.768

EPOCH  3 ...
Validation Accuracy = 0.804

EPOCH  4 ...
Validation Accuracy = 0.782

EPOCH  5 ...
Validation Accuracy = 0.833

EPOCH  6 ...
Validation Accuracy = 0.840

EPOCH  7 ...
Validation Accuracy = 0.872

EPOCH  8 ...
Validation Accuracy = 0.865

EPOCH  9 ...
Validation Accuracy = 0.829

EPOCH 10 ...
Validation Accuracy = 0.860

EPOCH 11 ...
Validation Accuracy = 0.850

EPOCH 12 ...
Validation Accuracy = 0.874

EPOCH 13 ...
Validation Accuracy = 0.880

EPOCH 14 ...
Validation Accuracy = 0.841

EPOCH 15 ...
Validation Accuracy = 0.881

EPOCH 16 ...
Validation Accuracy = 0.880

EPOCH 17 ...
Validation Accuracy = 0.883

EPOCH 18 ...
Validation Accuracy = 0.896

EPOCH 19 ...
Validation Accuracy = 0.885

EPOCH 20 .

In [ ]:
import random

visualize(org)